# Reduccion de la dimensionalidad con autoencoders

En esta parte de la experimentacion trabajaremos para poder reducir la dimension de los datos con aprendizaje profundo. Usaremos una arquitectura particular denominada 'autoencoder' (... desarrollar en el informe)

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.fashion_mnist.load_data()


4431872/4422102 [==============================] - 0s 0us/step


In [3]:

encoding_dim = 32

input_img = keras.Input(shape=(784,))
encoded = layers.Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(784, activation='sigmoid')(encoded)

# This amodel maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)
encoder = keras.Model(input_img, encoded)

encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = (autoencoder.layers[-1])
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))



In [4]:


autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

X_train = (X_train.astype('float32') / 255.)
X_val = (X_val.astype('float32') / 255.)
X_train = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))
X_val = X_val.reshape((len(X_val), np.prod(X_val.shape[1:])))
print(X_train.shape)
print(X_val.shape)



(60000, 784)
(10000, 784)


In [5]:
autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(X_val, X_val))

Epoch 1/50
235/235 [==============================] - 4s 12ms/step - loss: 0.4151 - val_loss: 0.3408
Epoch 2/50
235/235 [==============================] - 3s 12ms/step - loss: 0.3266 - val_loss: 0.3195
Epoch 3/50
235/235 [==============================] - 3s 11ms/step - loss: 0.3120 - val_loss: 0.3091
Epoch 4/50
235/235 [==============================] - 3s 12ms/step - loss: 0.3033 - val_loss: 0.3021
Epoch 5/50
235/235 [==============================] - 3s 12ms/step - loss: 0.2972 - val_loss: 0.2970
Epoch 6/50
235/235 [==============================] - 3s 12ms/step - loss: 0.2932 - val_loss: 0.2938
Epoch 7/50
235/235 [==============================] - 3s 12ms/step - loss: 0.2906 - val_loss: 0.2918
Epoch 8/50
235/235 [==============================] - 3s 12ms/step - loss: 0.2889 - val_loss: 0.2906
Epoch 9/50
235/235 [==============================] - 3s 12ms/step - loss: 0.2877 - val_loss: 0.2893
Epoch 10/50
235/235 [==============================] - 3s 12ms/step - loss: 0.2868 - val_lo

### Evaluacion con KNN

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

X_train_32 = encoder.predict(X_train)
X_val_32 = encoder.predict(X_val)


In [7]:
X_val_32.shape

(10000, 32)

In [8]:
model = KNeighborsClassifier(n_neighbors = 15).fit(X_train_32,y_train)

In [9]:
from sklearn.metrics import accuracy_score

preds = model.predict(X_val_32)

print(accuracy_score(y_val,preds))

0.8517


Muy buen resultado para una primera prueba.

## Redes mas profundas

Voy a hacer un modelo mas profundo para ver si el solo hecho de agregar profundidad rinde mejor.

In [19]:
input_img = keras.Input(shape=(784,))

encod_lvl1 = layers.Dense(392, activation='relu')(input_img)

encod_lvl2 = layers.Dense(200, activation='relu')(encod_lvl1)

encod_lvl3 = layers.Dense(100, activation='relu')(encod_lvl2)

neck = layers.Dense(32, activation='relu')(encod_lvl3)

decod_lvl3 = layers.Dense(100, activation='relu')(neck)

decod_lvl2 = layers.Dense(200, activation='relu')(decod_lvl3)

decod_lvl1 = layers.Dense(392, activation='relu')(decod_lvl2)

output_img = layers.Dense(784, activation='sigmoid')(decod_lvl1)


encoder = keras.Model(input_img,neck)
autoencoder = keras.Model(input_img,output_img)


In [20]:
autoencoder.compile(optimizer = 'adam' , loss ='binary_crossentropy')

autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(X_val, X_val))

Epoch 1/50
235/235 [==============================] - 9s 37ms/step - loss: 0.3684 - val_loss: 0.3167
Epoch 2/50
235/235 [==============================] - 9s 37ms/step - loss: 0.3059 - val_loss: 0.3018
Epoch 3/50
235/235 [==============================] - 9s 37ms/step - loss: 0.2967 - val_loss: 0.2952
Epoch 4/50
235/235 [==============================] - 9s 37ms/step - loss: 0.2915 - val_loss: 0.2921
Epoch 5/50
235/235 [==============================] - 9s 37ms/step - loss: 0.2884 - val_loss: 0.2912
Epoch 6/50
235/235 [==============================] - 9s 36ms/step - loss: 0.2858 - val_loss: 0.2870
Epoch 7/50
235/235 [==============================] - 9s 37ms/step - loss: 0.2837 - val_loss: 0.2856
Epoch 8/50
235/235 [==============================] - 9s 37ms/step - loss: 0.2822 - val_loss: 0.2834
Epoch 9/50
235/235 [==============================] - 9s 37ms/step - loss: 0.2808 - val_loss: 0.2827
Epoch 10/50
235/235 [==============================] - 9s 37ms/step - loss: 0.2796 - val_lo

In [21]:
X_train_red_deep = encoder.predict(X_train)

In [22]:
X_val_red_deep = encoder.predict(X_val)
knn = KNeighborsClassifier(n_neighbors=15).fit(X_train_red_deep,y_train)

preds = knn.predict(X_val_red_deep)

accuracy_score(y_val,preds)

0.8639

# Luego
# Experimentacion con multiples profundidades